## Mail Sender

It sends an email every 10 seconds, from 7:00 AM to 6:00 PM, Monday through Friday. The emails are retrieved from a database, and the system keeps track of which emails have already been sent. If the process is interrupted, it resumes sending emails from where it left off.

In [ ]:
import os
import pandas as pd
from datetime import datetime
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage
from email.mime.application import MIMEApplication
from email import encoders
import base64
import time
import csv

In [ ]:
SCOPES = ['https://www.googleapis.com/auth/gmail.send']

In [ ]:
# Initialization and authorization of Gmail client
def authorize():
    credentials = None
    # Checking if a token exists (generated after the first login)
    if os.path.exists('token.json'):
        credentials = Credentials.from_authorized_user_file('token.json', SCOPES)
    # Authorization if the token is invalid or does not exist
    if not credentials or not credentials.valid:
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            credentials = flow.run_local_server(port=8080)
        # Saving token for future use
        with open('token.json', 'w') as token:
            token.write(credentials.to_json())
    return credentials


# Function to send an email with an attachment and image
def send_email(to, subject, message_content, pdf_path=None, image_path=None):
    credentials = authorize()
    service = build('gmail', 'v1', credentials=credentials)
    
    # Set up the MIME message
    message = MIMEMultipart()
    message['From'] = "me"
    message['To'] = to
    message['Subject'] = subject
    
    # Adding HTML content
    html_content = MIMEText(message_content, 'html')
    message.attach(html_content)
    
    # Adding an image as an inline attachment
    if image_path:
        with open(image_path, 'rb') as f:
            img = MIMEImage(f.read())
            img.add_header('Content-ID', '<logo>')  # Setting the cid:logo identifier
            img.add_header('Content-Disposition', 'inline', filename=os.path.basename(image_path))
            message.attach(img)

    # Adding PDF as an attachment
    if pdf_path:
        with open(pdf_path, 'rb') as attachment:
            mime_base = MIMEBase('application', 'pdf')
            mime_base.set_payload(attachment.read())
            encoders.encode_base64(mime_base)
            mime_base.add_header('Content-Disposition', f'attachment; filename="{os.path.basename(pdf_path)}"')
            message.attach(mime_base)
    
    # Encode message and prepare it for sending
    raw_message = base64.urlsafe_b64encode(message.as_bytes()).decode()
    body = {'raw': raw_message}
    
    # Sending message via Gmail API
    try:
        sent_message = service.users().messages().send(userId='me', body=body).execute()
        print(f"E-mail sent to {to} with message ID: {sent_message['id']}")
    except Exception as error:
        print(f"An error occurred: {error}")


def time_to_send():
    now = datetime.now()
    return now.weekday() < 5 and 7 <= now.hour < 18

def load_sent():
    if os.path.exists('sent_emails.csv'):
        with open('sent_emails.csv', mode='r') as file:
            sent = {line.strip() for line in file}
    else:
        sent = set()
    return sent

def save_sent(address):
    with open('sent_emails.csv', mode='a') as file:
        file.write(f"{address}\n")

def save_progress(last_address):
    with open('sending_status.txt', 'w') as file:
        file.write(last_address)

def load_progress():
    if os.path.exists('sending_status.txt'):
        with open('sending_status.txt', 'r') as file:
            return file.read().strip()
    return None

def send_emails_from_database(excel_path, subject, content, pdf_path=None, image_path=None):
    df = pd.read_excel(excel_path)
    email_addresses = df['E-mail'].dropna().tolist()
    
    sent = load_sent()
    last_address = load_progress()
    start = not last_address  # Start true if there is no last address

    for address in email_addresses:
        if address == last_address:  # Found the address from which to continue
            start = True
        if not start or address in sent:  # Skip already sent addresses
            continue

        if time_to_send():
            try:
                # Adding logo to the content of the email
                content_with_logo = content.replace('cid:logo.png', 'cid:logo')
                send_email(address, subject, content_with_logo, pdf_path, image_path)
                save_sent(address)
                save_progress(address)
                print(f"E-mail sent to: {address}")
                time.sleep(10)
            except Exception as e:
                print(f"Failed to send email to {address}: {e}")
                save_progress(address)  # Save progress if error occurs
        else:
            print("Outside working hours. Sending paused.")
            save_progress(address)  # Save the last sent address
            break

In [ ]:
# Przykład użycia
send_emails_from_database(
    r'C:\Users\User\Jupyter_praca\MailSender\MaileKsiazki.xlsx',
    'BookWorld - Nowy katalog książek',
    '''
    <p>Szanowni Państwo,</p>
    
    <p>Z przyjemnością przedstawiamy Państwu nasz najnowszy katalog książek, przygotowany z myślą o miłośnikach literatury. Znajdziecie w nim zarówno klasyczne dzieła, jak i najnowsze premiery, które mogą wzbogacić Państwa bibliotekę lub zainteresować klientów.</p>
    
    <p><strong>Dlaczego warto wybrać BookWorld?</strong></p>
    
    <ul>
        <li><strong>Różnorodność:</strong> Tysiące tytułów z różnych gatunków literackich.</li>
        <li><strong>Jakość:</strong> Współpracujemy z najlepszymi wydawnictwami.</li>
        <li><strong>Ekspercka obsługa:</strong> Nasz zespół służy pomocą w wyborze idealnych pozycji.</li>
        <li><strong>Szybka dostawa:</strong> Książki dostarczamy bezpośrednio pod Państwa drzwi.</li>
    </ul>
    
    <p>Zachęcamy do pobrania naszego katalogu i zapoznania się z pełną ofertą. W przypadku pytań lub dodatkowych informacji – pozostajemy do Państwa dyspozycji!</p>
    
    <div style="display: flex; align-items: center; justify-content: space-between;">
        <div>
            <p>Z poważaniem,<br>
            <strong>Zespół BookWorld</strong><br>
            <a href="https://BookWorld.pl">BookWorld.pl</a><br>
            <a href="mailto:kontakt@bookworld.com">kontakt@bookworld.com</a><br>
            22 123 45 67<br>
            22 123 45 89</p>
        </div>
        <div style="float: right; text-align: right; margin-top: 2em;">
            <img src="cid:logo.png" width="150" height="75" alt="Logo firmy" style="margin-left: 20px;">
        </div>
    </div>

    <p>Wiadomość wygenerowana automatycznie. Prosimy nie odpowiadać.</p>
    ''',
    r'C:\Users\User\Jupyter_praca\MailSender\Katalog_BookWorld.pdf',
    image_path = r'C:\Users\User\Jupyter_praca\MailSender\logo.png'  
)